For this lab, you are going to be combining the fare data you used in the first homework with some trip data about those same exact taxi rides.

Here is the schema of the trip dataset, found in `trip_data_500k.csv`:

* `medallion`: The ID of the cab being operated
* `hack_license`: The ID of the person operating the cab
* `vendor_id`: The type of vendor operating the cab, can either be `CMT` or `VTS`, no clue what these two types mean
* `rate_code`: Designates the kind of ride this is, must be `1` through `6`, any other number is incorrect
* `store_and_fwd_flag`: Can be either `Y`,`N`, or Nan
* `pickup_datetime`: The time when the ride started
* `dropoff_datetime`: The time when the ride ended
* `passenger_count`: The number of passengers during the ride
* `trip_time_in_secs`: How long the trip took
* `trip_distance`: Distance of the trip, to the nearest 1/10 mile
* `pickup_longitude`: Longitude of pickup location
* `pickup_latitude`: Latitude of pickup location
* `dropoff_longitude`: Longitude of dropoff location
* `dropoff_latitude`: Latitude of dropoff location

And here is the schema for the fare dataset, as a refresher (you're going to be using it too :)), found in `trip_fare_500k.csv`:

* `medallion`: The ID of the cab being operated
* `hack_license`: The ID of the person operating the cab
* `vendor_id`: The type of vendor operating the cab, can either be `CMT` or `VTS`, no clue what these two types mean
* `pickup_datetime`: The time when the ride started
* `payment_type`: How the trip was paid, `UNK` stands for unknown, I have no idea what `NOC` stands for, but lets assume its some known way to pay
* `fare_amount`: Base fare cost of the trip
* `surcharge`: Additional charges that are not tolls
* `mta_tax`: The mta has to get its cut, right? :)
* `tip_amount`: How generous the rider(s) decided to be
* `tolls_amount`: How much money you had to pay in tolls
* `total_amount`: How much the trip cost, all in

One final piece of information you will need, the approximate latitude/longitude bounds of a bounding box centered around each borough in NYC:

* Queens,40.800760,40.542920,-73.700272,-73.962616
* Manhattan,40.874663,40.701293,-73.910759,-74.018721
* Bronx,40.915255,40.785743,-73.765274,-73.933406
* Brooklyn,40.739877,40.57042,-73.864754,-74.04344
* Staten Island,40.651812,40.477399,-74.034547,-74.259090

So, as was the case with the last lab, you are going to be tasked with answering a whole slew of questions about these data, except the questions here should be significantly more challenging.

Before beginning, please:

1. Remove any rides that did not have an appropriate `rate_code` 
* Convert pickup and dropoff locations from latitude/longitude to (approximate) borough. This will be very challening. I suggest the following approach, although you can try others:
  1. Calculate the exact center of each borough's bounding box and store each in a new variable (you should have one of these per borough, so 5 variables)
  * Find the distance from the given ride's pickup/dropoff location and the center of each borough found in A. 
  * Pick the borough that had the smallest distance from the given location to its bounding box center. (this is messy because it ignores cases where the start/end location isn't in the 5 boroughs, but its the best we can do with the information you're provided)
* Once this is done, join the `trip_data` and `fare_data` datasets together. You will need to join the datasets on more than one column, but you will have to figure out what those columns are!

Once you've gotten the preprocessing out of the way, answer the following questions:

1. What was the most common borough start location?
  * End location?
  * Pair of start/end locations? Excluding manhattan/manhattan?
* Which driver (`hack_license`) carried the most passengers, on average?
* Which driver had the highest tip percentage, on average, for those drivers that made at least 5 trips in the dataset?
* Was there any relationship between (correlation) when a taxi ride ended (get the closest minute within the day) and the tip percentage on the fare?
* Did the trip time correlate with the cost of the trip?
  * What about the tip amount?
* Which cab tended to generate the most revenue, on average, when picking people up in Manhattan?
* What was the average cost of all of the trips that originated in Brooklyn?
* Which driver made the most money overall? Assume the only money made was from tips.
  * Which driver made the most money on average?
  * Which driver made the most money in each borough?
* What was the average trip distance and trip cost for intra-borough (same starting/ending borough) trips?
  1. For inter-borough (different starting/ending borough) trips?
* Which borough had the cheapest tippers (had the smallest average tip percentage)? Assume that if a trip starts within some borough, then that trip belongs in that borough.
* Which driver logged the most miles in this dataset?
* What was the average toll amount for intra-borough rides? For inter-borough  rides?

In [1]:
import pandas as pd
import numpy as np
from geopy.distance import great_circle

In [2]:
queensCenter = ((40.800760+40.542920)/2,(-73.700272-73.962616)/2)
brookCenter = ((40.739877+40.57042)/2,(-73.864754-74.04344)/2)
bronxCenter = ((40.915255+40.785743)/2,(-73.765274-73.933406)/2)
manhattanCenter = ((40.874663+40.701293)/2,(-73.910759-74.018721)/2)
siCenter = ((40.651812+40.477399)/2,(-74.034547-74.259090)/2)
boroughDict = {}
boroughDict["queens"] = queensCenter
boroughDict["brooklyn"] = brookCenter
boroughDict["bronx"] = bronxCenter
boroughDict["manhattan"] = manhattanCenter
boroughDict["staten"] = siCenter

#This function returns the string representation of the likely borough, given a set of latitude/longitude coordinates
#If the distance to the borough center is too far away from the closest borough, we assume that the location
#is outside of NYC
def get_closest_borough(latitude,longitude,max_dist = 20):
    global boroughDict
    borough_distances = {borough:great_circle(boroughDict[borough],(latitude,longitude)).miles for borough in boroughDict}
    min_borough = min(borough_distances, key=borough_distances.get)
    if borough_distances[min_borough] < max_dist:
        return min_borough 
    else:
        return "outside_nyc"

In [4]:
trips = pd.read_csv("../data/nycTaxiData/trip_data_500k.csv")
fares = pd.read_csv("../data/nycTaxiData/trip_fare_500k.csv")

In [13]:
#remove bad records from the dataset as some latitudes were not correctly reported
trips = trips[trips.pickup_latitude<=90]
trips = trips[trips.pickup_latitude>=-90]
trips = trips[trips.dropoff_latitude>=-90]
trips = trips[trips.dropoff_latitude<=90]

In [14]:
trips["pickup_borough"] = trips.apply(\
                        lambda x:\
                                get_closest_borough(x['pickup_latitude'],
                                                    x['pickup_longitude']),
                                      axis=1)
trips["dropoff_borough"] = trips.apply(\
                        lambda x:\
                                get_closest_borough(x['dropoff_latitude'],
                                                    x['dropoff_longitude']),
                                      axis=1)

In [5]:
full_data = trips.merge(fares,on=["medallion","hack_license","vendor_id","pickup_datetime"])
del full_data["pickup_latitude"]
del full_data["pickup_longitude"]
del full_data["dropoff_longitude"]
del full_data["dropoff_latitude"]

In [6]:
# 1. What was the most common borough start location?
print(full_data.pickup_borough.value_counts().index[0])
#   * End location?
print(full_data.dropoff_borough.value_counts().index[0])

manhattan
manhattan


In [7]:
#   * Pair of start/end locations?
pick_drop_comb_sizes = full_data.groupby(["pickup_borough","dropoff_borough"]).size()
pick_drop_comb_sizes.sort_values(ascending=False,inplace=True)
print(pick_drop_comb_sizes.index[0])
# Excluding manhattan/manhattan?
print(pick_drop_comb_sizes[~pick_drop_comb_sizes.index.isin([("manhattan","manhattan")])].index[0])

('manhattan', 'manhattan')
('manhattan', 'brooklyn')


In [8]:
# * Which driver (`hack_license`) carried the most passengers, on average?
pass_per_hack = full_data.groupby("hack_license")["passenger_count"].mean()
pass_per_hack.sort_values(ascending=False,inplace=True)
pass_per_hack[pass_per_hack==pass_per_hack.max()]

hack_license
DF1338A98DAA39B20B528EEC54081A3D    6.0
520B738CBE7F00D4E2B0BC20EB10F632    6.0
0A877DA2820B8CB9EF54702D80609A6F    6.0
F7520959F030ECCD11C85F3B7C23EB4D    6.0
8B6D35967E3EF52C75E0E97F7DC8C13E    6.0
1FA8048B64BB78D6989C17D8FA89833E    6.0
458D14DB813121C1CE7AB9F3956959FF    6.0
BCBC4DC29AA98BEEBE373FDCBA69A0C1    6.0
66D0CB86A84D35070AA1ABC3F1BFBC2A    6.0
BC76B34FA77F0FA0B931827A847815AC    6.0
8BF2F2FB341011F4B50A3F617342E535    6.0
C73CC5B4772C0803021069CFBB259510    6.0
BC06BC4CB757E3F09B1F65C055ADC51B    6.0
F7299385AADB0C60DF07149C3239B7C7    6.0
F703AE4563AA348787DB26458372E5F3    6.0
35238090BD3CE4ACF47FEBFDD023A381    6.0
205994280475874B569FC4213BC36605    6.0
BB417F8F75B387F11F4AA9BD2359A3B0    6.0
BAEEBF42684943FEE0D2C227E46A934F    6.0
8D13DF4AFDE3F36B931990502191E5DE    6.0
BD40831DBE6C587175967E26DC9219EF    6.0
F76223B653DA42726DC7D41F32F08F4E    6.0
8AF8551CD00D18548E3C723BBAEDF414    6.0
5C8BAF38B31932AD0D52C43107716FCD    6.0
BE381B7969C0BC767DCDA7974BF

In [9]:
full_data["tip_percent"] = full_data.tip_amount/full_data.fare_amount

In [12]:
# * Which driver had the highest tip percentage, on average,
#for those drivers that made at least 5 trips in the dataset?
atleast_5_rides = full_data.groupby("hack_license").filter(lambda x: x.shape[0]>5)
driver_tips = atleast_5_rides.groupby("hack_license")["tip_percent"].mean()
driver_tips.sort_values(ascending=False,inplace=True)
driver_tips.head(1)

hack_license
3B8AC94620C43B364A42B3A7D1439B68    5.888636
Name: tip_percent, dtype: float64

In [25]:
# * Was there any relationship between (correlation) when a taxi ride ended 
#(get the closest minute within the day) and the tip percentage on the fare?
full_data.dropoff_datetime = pd.to_datetime(full_data.dropoff_datetime)
(full_data.dropoff_datetime.dt.hour * 60 + full_data.dropoff_datetime.dt.minute).corr(full_data.tip_percent)

0.015118706841059299

In [27]:
# * Did the trip time correlate with the cost of the trip?
print(full_data.trip_time_in_secs.corr(full_data.total_amount))
#   * What about the tip amount?
full_data.trip_time_in_secs.corr(full_data.tip_amount)

0.789873169267


0.44367923374265394

In [31]:
# * Which cab tended to generate the most revenue, on average, when picking people up in Manhattan?
avg_per_manh_cab_revenue = full_data[full_data.pickup_borough=='manhattan'].groupby("medallion")["total_amount"].mean()
avg_per_manh_cab_revenue[avg_per_manh_cab_revenue==avg_per_manh_cab_revenue.max()]

medallion
9AE01AEEA2E4D0A9BBDA62946FCE81D0    84.0
Name: total_amount, dtype: float64

In [32]:
# * What was the average cost of all of the trips that originated in Brooklyn?
per_borough_pickup_avg_cost = full_data.groupby("pickup_borough")["total_amount"].mean()
per_borough_pickup_avg_cost[per_borough_pickup_avg_cost.index == 'brooklyn']

pickup_borough
brooklyn    16.172874
Name: total_amount, dtype: float64

In [35]:
# * Which driver made the most money overall? Assume the only money made was from tips.
per_driver_revenue = full_data.groupby("hack_license")["tip_amount"].sum()
per_driver_revenue.sort_values(ascending=False, inplace =True)
per_driver_revenue.head(1)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
  app.launch_new_instance()


hack_license
BD0913D639AA03DA954EA97E2A3A1101    225.34
Name: tip_amount, dtype: float64

In [36]:
#   * Which driver made the most money on average?
per_driver_avg_revenue = full_data.groupby("hack_license")["tip_amount"].mean()
per_driver_avg_revenue.sort_values(ascending=False, inplace =True)
per_driver_avg_revenue.head(1)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
  app.launch_new_instance()


hack_license
3E8BC9829EE46234B580C2DA5ED69C0C    30.69
Name: tip_amount, dtype: float64

In [45]:
#   * Which driver made the most money in each borough?
per_driver_borough_revenue = full_data.groupby(["pickup_borough","hack_license"])["tip_amount"].sum().reset_index()
per_driver_borough_revenue.groupby("pickup_borough").apply(lambda x: x[["hack_license","tip_amount"]][x.tip_amount==x.tip_amount.max()])

,,hack_license,tip_amount
pickup_borough,,,
bronx,57,427DE707D035285C19D538711B6E36FF,54.68
brooklyn,882,0F8CD038D26AB9254A8566FC646B3157,83.80
manhattan,14298,44FBBE4EFBE80F8CA9E3C1CD88A5619A,205.51
outside_nyc,25476,3B8AC94620C43B364A42B3A7D1439B68,160.00
queens,30853,E6CB1818691BA4D223F1E148CE727864,109.80
staten,31318,317799A8C9027ED5E9CBB1DEB38238D9,87.00


In [53]:
# * What was the average trip distance and trip cost for intra-borough (same starting/ending borough) trips?
print("Trip distance and cost intraborough:\n",full_data[full_data.pickup_borough==
                                            full_data.dropoff_borough][["trip_distance",
                                                                        "total_amount"]].mean(),"\n")
#   1. For inter-borough (different starting/ending borough) trips?
print("Same but for interborough rides:\n",full_data[full_data.pickup_borough!=
                                                     full_data.dropoff_borough][["trip_distance",
                                                                                 "total_amount"]].mean(),"\n")
# * Which borough had the cheapest tippers (had the smallest average tip percentage)? Assume that if a trip starts within some borough, then that trip belongs in that borough.
print("Cheapest tippers:\n",full_data.groupby(["dropoff_borough"])["tip_percent"].mean(),"\n")
# * Which driver logged the most miles in this dataset?
per_driver_mileage = full_data.groupby(["hack_license"])["trip_distance"].sum()
per_driver_mileage.sort_values(ascending=False,inplace=True)
print(per_driver_mileage.head(1),"\n")
# * What was the average toll amount for intra-borough rides? For inter-borough  rides?
print("Toll amount intraborough:\n",full_data[full_data.pickup_borough==
                                            full_data.dropoff_borough][["tolls_amount"]].mean(),"\n")
print("Toll amount interborough:\n",full_data[full_data.pickup_borough!=full_data.dropoff_borough][["tolls_amount"]].mean())

Trip distance and cost intraborough:
trip_distance     2.175694
total_amount     11.928908
dtype: float64 

Same but for interborough rides:
trip_distance     7.188409
total_amount     28.078621
dtype: float64 

Cheapest tippers:
dropoff_borough
bronx          0.073805
brooklyn       0.109801
manhattan      0.105904
outside_nyc    0.137292
queens         0.087671
staten         0.171467
Name: tip_percent, dtype: float64 

hack_license
15589D617DB1B13418DE284711599A7D    453.77
Name: trip_distance, dtype: float64 

Toll amount intraborough:
tolls_amount    0.178392
dtype: float64 

Toll amount interborough:
tolls_amount    0.78012
dtype: float64


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
